In [ ]:
# 기사 목록

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse


# 크롬 드라이버 옵션 설정
chrome_options = Options()
# chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로드 비활성화
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 검색어, 시작 날짜, 종료 날짜 입력
keywords = [
    "검색어",
]
start_date = "2024-06-01"
end_date = "2024-08-31"

collected_urls = set()

# 크롬 드라이버 설정
driver = webdriver.Chrome(options=chrome_options)

def search_and_save_articles(keyword1, keyword2, start_date, end_date):
    base_url = "https://www.daejonilbo.com/news/articleList.html?page="
    results = []
    page = 1

    while True:
        
        print(f"{keyword1} {keyword2} {page} 수집 중...")
        search_url = base_url + f"{page}&box_idxno=&period=all&sc_sdate={start_date}&sc_edate={end_date}&sc_section_code=&sc_area=A&sc_word=" + urllib.parse.quote(keyword1) +"&sc_andor=AND&sc_word2=" + urllib.parse.quote(keyword2)

        driver.get(search_url)
        
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul.type")))
        except Exception as e:
            print(f"페이지 로딩 중 오류 발생: {e}")
            break

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기사 목록 가져오기
        articles = soup.select('ul.type > li')
        new_articles_found = False  # 새로운 기사가 발견되었는지 확인하는 플래그

        # 기사 정보 추출
        for article in articles:
            try:
                title_tag = article.select_one('h4.titles')
                link_tag = article.select_one('h4.titles a')
                byline = article.select_one('span.byline')
                
                title = title_tag.get_text(strip=True) if title_tag else ""
                url = "https://www.daejonilbo.com" + link_tag['href'] if link_tag else ""
                date = byline.get_text(strip=True).split()[0] if byline else ""

                if url and url not in collected_urls:
                    collected_urls.add(url)
                    new_articles_found = True  # 새로운 기사가 발견됨

                    if not new_articles_found:  # 새로운 기사가 발견되지 않으면 종료
                        break
                
                    # 결과 리스트에 추가
                    results.append({
                        'media': '대전일보', 
                        'title': title, 
                        'date': date, 
                        'section': '', 
                        'article_type': '', 
                        'writer': '', 
                        'page': '', 
                        'url': url, 
                        'content': ''  # 컨텐츠 정보 없음
                    })
            except Exception as e:
                print(f"기사 처리 중 오류 발생: {e}")



        # 다음 페이지로 이동
        page += 1
                          
        return results

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)

all_results = []

try:
    for keyword in keywords:
        keyword_split = keyword.split()  # 키워드를 공백 기준으로 분리
        keyword1 = keyword_split[0] if len(keyword_split) > 0 else ''  # 첫 번째 키워드
        keyword2 = keyword_split[1] if len(keyword_split) > 1 else ''  # 두 번째 키워드 (없으면 빈 문자열)
        all_results.extend(search_and_save_articles(keyword1, keyword2, start_date, end_date))

    df = pd.DataFrame(all_results)

    # 기사 수 출력
    print(f"Total number of articles: {len(df)}")

    # 중복 제거된 데이터 저장
    save_to_excel(df, f'2대전일보 {start_date}~{end_date}.xlsx')

finally:
    # 드라이버 종료
    driver.quit()


In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문, 서브 타이틀 및 작성자 정보를 스크랩하는 함수 정의
def get_content_subtitle_and_writer_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 대전일보
        article_body = soup.find('div', class_="article-body")
        writer = ""
        subtitle = ""

        if article_body:
            # 'caption' 클래스를 가진 모든 태그를 제거합니다.
            for caption in article_body.find_all('div', style='text-align:center'):
                caption.decompose()

            # 서브 타이틀 추출 (h4.subheading)
            subtitle_tag = soup.find('h4', class_='subheading')
            if subtitle_tag:
                subtitle = subtitle_tag.get_text(separator='\n', strip=True)
                subtitle_tag.decompose()  # 서브 타이틀 본문에서 제거

            # 작성자 정보 추출
            writer_section = soup.find('article', class_='writer')
            if writer_section:
                writer_span = writer_section.find('span', class_='name')
                if writer_span:
                    for a_tag in writer_span.find_all('a'):
                        a_tag.decompose()  # 이메일 제거
                    writer = writer_span.get_text(strip=True)

                next_sibling = writer_section.find_next_sibling()
                while next_sibling:
                    next_sibling.decompose()
                    next_sibling = writer_section.find_next_sibling()
                writer_section.decompose()  # writer 자체 제거

            # 나머지 본문 텍스트 추출
            content = article_body.get_text(separator='\n', strip=True)
        else:
            content = ""
        
        return content, subtitle, writer
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None, None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문, 서브 타이틀, 작성자 정보를 스크랩하여 저장
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle, writer = get_content_subtitle_and_writer_from_url(row['url'])
    
    if content is None:
        error_log.append(row['url'])
    
    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle  # 서브 타이틀 추가
    df.at[index, 'writer'] = writer

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 오류 로그 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")

# 변경된 데이터를 파일로 저장
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")
